In [ ]:
import sys
sys.path.append('../utils')
sys.path.append('../core')
sys.path.append('../')
import numpy as np
import scipy
import tqdm
import utils
import geometry
from core import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import glob

from midvoxio.voxio import vox_to_arr

In [ ]:
blood_mask = vox_to_arr('../experiment_files/images/blood.vox')
myocardium_mask = vox_to_arr('../experiment_files/images/myocardium.vox')
blood_mask = np.where(np.sum(blood_mask, axis=-1) > 0, 1, 0)
myocardium_mask = np.where(np.sum(myocardium_mask, axis=-1) > 0, 1, 0)

In [ ]:
test_phantom = phantom.Phantom(source_path = None,
                               voxel_dims = (0.5e-3,0.5e-3,0.5e-3),
                               matrix_dims = (256,256,256),
                               baseline = (1540, 1000),
                               seed = 5678,
                               )

blood = tissue.Tissue(name='blood', c=1578, rho=1060, sigma=1.3, scale=0.00001, label=1)
myocardium = tissue.Tissue(name='myocardium', c=1592, rho=1081, sigma=20, scale=0.0001, label=2)
fat = tissue.Tissue(name='fat', c=1450, rho=920, sigma=10, scale=0.0001, label=3)

test_phantom.add_tissue(fat, mask=np.ones((256,256,256)))
test_phantom.add_tissue(myocardium, mask=myocardium_mask)
test_phantom.add_tissue(blood, mask=blood_mask)
test_phantom.set_default_tissue('fat')
# plt.imshow(test_phantom.get_complete()[0,:,:,80])
# test_phantom.save('../experiment_files/cardiac_phantom')
# utils.save_mrc(test_phantom.get_complete(), '../experiment_files/cardiac_phantom/cardiac_phantom.mrc')

In [ ]:
plt.imshow(test_phantom.get_complete()[0,:,:,test_phantom.matrix_dims[2]//2])

In [ ]:
transducers = [transducer.Focused(max_frequency = 2e6,
                                  elements = 64, 
                                  width = 20e-3,
                                  height =  20e-3,
                                  sensor_sampling_scheme = 'not_centroid', 
                                  sweep = np.pi/2,
                                  ray_num = 64,
                                  imaging_ndims = 2,
                                  focus_azimuth = 100e-3,
                                  focus_elevation = 100e-3,
                                  cycles = 3,
) for i in range(7)]

for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

In [ ]:
test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

# apical 4 chamber
transform = geometry.Transform([-np.pi*0.28,-np.pi*0.18,-np.pi*0.05], [-.052,0.055,-0.072])
test_transducer_set.assign_pose(0, transform)

# apical 2 chamber
transform = geometry.Transform([-np.pi*0.34,-np.pi*0.18,-np.pi*0.35], [-.052,0.055,-0.072])
test_transducer_set.assign_pose(1, transform)

# parasternal short axis mitral
transform1 = geometry.Transform([-np.pi/2 * 0.2,0,0], [0,0,0])
transform2 = geometry.Transform([0,np.pi/2 * 0.35,0], [0,0,0])
transform3 = geometry.Transform([-np.pi/2 * 0.5,0,0], [0,0,0])
transform = transform1 * transform2 * transform3
position = [-0.05,0,-0.020]
transform.translation = transform.apply_to_point(position, inverse=False)
test_transducer_set.assign_pose(2, transform)

# parasternal shout axis papillary
transform1 = geometry.Transform([-np.pi/2 * 0.2,0,0], [0,0,0])
transform2 = geometry.Transform([0,np.pi/2 * 0.75,0], [0,0,0])
transform3 = geometry.Transform([-np.pi/2 * 1,0,0], [0,0,0])
transform = transform1 * transform2 * transform3
position = [-0.05, 0.02, -0.045]
transform.translation = transform.apply_to_point(position, inverse=False)
test_transducer_set.assign_pose(3, transform)

# parasternal short axis aortic
transform1 = geometry.Transform([-np.pi/2 * 0,0,0], [0,0,0])
transform2 = geometry.Transform([0,np.pi/2 * 0.20,0], [0,0,0])
transform3 = geometry.Transform([-np.pi/2 * 0.5,0,0], [0,0,0])
transform = transform1 * transform2 * transform3
position = [-0.05,0,-0.011]
transform.translation = transform.apply_to_point(position, inverse=False)
test_transducer_set.assign_pose(4, transform)

# parasternal long axis
transform1 = geometry.Transform([0,0,0], [0,0,0])
transform2 = geometry.Transform([0,-np.pi/2 * 0.55,0], [0,0,0])
transform3 = geometry.Transform([-np.pi * 0.6,0,0], [0,0,0])
transform = transform1 * transform2 * transform3
position = [-0.05,-0.01,-0.0010]
transform.translation = transform.apply_to_point(position, inverse=False)
test_transducer_set.assign_pose(5, transform)

# subcostal 4 chamber
transform1 = geometry.Transform([-np.pi*0.28,0,0], [0,0,0])
transform2 = geometry.Transform([0,-np.pi*0.18,0], [0,0,0])
transform3 = geometry.Transform([0,0,-np.pi*0.05], [0,0,0])
transform4 = geometry.Transform([-np.pi * 0.2,0,0], [0,0,0])
transform = transform1 * transform2 * transform3 * transform4
position = [-0.09,-0.01,-0.023]
transform.translation = transform.apply_to_point(position, inverse=False)
test_transducer_set.assign_pose(6, transform)

# # subcostal 4 chamber
# transform1 = geometry.Transform([-np.pi*0.28,0,0], [0,0,0])
# transform2 = geometry.Transform([0,-np.pi*0.18,0], [0,0,0])
# transform3 = geometry.Transform([0,0,-np.pi*0.05], [0,0,0])
# transform4 = geometry.Transform([-np.pi * 0.2,0,0], [0,0,0])
# transform = transform1 * transform2 * transform3 * transform4
# position = [-0.09,-0.01,-0.023]
# transform.translation = transform.apply_to_point(position, inverse=False)
# test_transducer_set.assign_pose(7, transform)



# test_transducer_set.plot_transducer_fovs(scale=0.07, view=(90,0))
# test_transducer_set.plot_transducer_fovs(scale=0.07, view=(0,0))
# test_transducer_set.plot_transducer_fovs(scale=0.07, view=(0,90))
test_transducer_set.plot_transducer_fovs(scale=0.1, view=(30,45))
test_transducer_set.plot_transducer_coords(scale=0.1, view=(30,45))

In [ ]:
# test_sensor = sensor.Sensor(transducer_set=test_transducer_set, aperture_type='pressure_field')
test_sensor = sensor.Sensor(transducer_set=test_transducer_set, aperture_type='transmit_as_receive')

In [ ]:
simprops = simulation.SimProperties(
                grid_size   = (150e-3,50e-3,50e-3),
                voxel_size  = (0.5e-3,0.5e-3,0.5e-3),
                PML_size    = (32,8,8),
                PML_alpha   = 2,
                t_end       = 12e-5,           # [s]
                bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                alpha_coeff = 0.5, 	           # [dB/(MHz^y cm)]
                alpha_power = 1.5,
                )

In [ ]:
test_experiment = experiment.Experiment(
                 simulation_path = '../experiment_files/cardiac_TTE_partial',
                 sim_properties  = simprops,
                 phantom         = test_phantom,
                 transducer_set  = test_transducer_set,
                 sensor          = test_sensor,
                 nodes           = 1,
                 results         = None,
                 indices         = None,
                 workers         = 3,
                 additional_keys = []
                 )

test_experiment.save()

In [ ]:
t = 0

In [ ]:
test_experiment.plot_ray_path(16 + (64 * t))

In [ ]:
test_experiment.plot_ray_path(32 + (64 * t))

In [ ]:
test_experiment.plot_ray_path(48 + (64 * t))

In [ ]:
test_experiment = experiment.Experiment.load('../experiment_files/cardiac_TTE_partial')
test_experiment.run(dry=True)

In [ ]:
test_experiment.run(repeat=False)

In [ ]:
test_experiment.add_results()

In [ ]:
test_reconstruction = reconstruction.DAS(experiment=test_experiment)

In [ ]:
# image, signals = test_reconstruction.get_image(dimensions=3, matsize=100, downsample=0.05, workers=1)

In [ ]:
signals = test_reconstruction.get_signals(dimensions=2, matsize=256, downsample=1, workers=1, tgc=7)

In [ ]:
plt.imshow(signals[0], vmin=0, vmax=12000, cmap='gray')
plt.gca().set_facecolor('k')

In [ ]:
image = np.where(np.isnan(image), 0, image)

In [ ]:
np.sum(image)

In [ ]:
utils.save_mrc(image, '../experiment_files/cardiac_experiment_04/image.mrc')

In [ ]:
X = np.linspace(-1, 1, 256)
Y = np.linspace(-1, 1, 256)
X, Y = np.meshgrid(X, Y)

fig, ax = plt.subplots(2,4, figsize=(10,5))
ax[0,0].pcolormesh(X, Y, signals[0], shading='auto', cmap='gray', norm='log')
# ax[0,1].pcolormesh(X, Y, signals[1], shading='auto', cmap='gray', norm='log')
# ax[0,2].pcolormesh(X, Y, signals[2], shading='auto', cmap='gray', norm='log')
# ax[0,3].pcolormesh(X, Y, signals[3], shading='auto', cmap='gray', norm='log')
# ax[1,0].pcolormesh(X, Y, signals[4], shading='auto', cmap='gray', norm='log')
# ax[1,1].pcolormesh(X, Y, signals[5], shading='auto', cmap='gray', norm='log')
# ax[1,2].pcolormesh(X, Y, signals[6], shading='auto', cmap='gray', norm='log')
# ax[1,3].pcolormesh(X, Y, signals[7], shading='auto', cmap='gray', norm='log')

for axd1 in ax:
    for a in axd1:
        a.set_aspect('equal')
        a.set_xticks([])
        a.set_yticks([])

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.pcolormesh(X, Y, image, shading='auto', cmap='gray', norm='log')
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
fig, ax = plt.subplots(1, 1)
image_rays = np.stack([np.mean(test_experiment.results[i][1], axis=0)[100:] for i in range(len(test_experiment.results))])
ax.pcolor(np.log(image_rays + 2))

In [ ]:
im = test_reconstruction.results[0][2]

In [ ]:
im2 = im.reshape((1232,112,-1))

In [ ]:
plt.imshow(im2)

In [ ]:
im.shape

In [ ]:
plt.imshow(im[:,:,300].T, cmap='bwr')

In [ ]:
for frame in tqdm.tqdm(range(0,im.shape[-1],10)):
    plt.imshow(im[:,:,frame], cmap='bwr')
    plt.colorbar()
    plt.savefig(f'../experiment_files/outputs/frame_{str(frame).zfill(4)}.png', dpi=50)
    plt.close()